In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Dependencies and Setup FOR API
import numpy as np
import requests
import time
from datetime import datetime
import json

# Import API key
from api_keys import tmdb_api_key





### Store CSV into DataFrame

In [2]:
disney_csv_file = "./Datasets/Disney_titles.csv"
disney_titles_df = pd.read_csv(disney_csv_file)
disney_titles_df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm74391,Fantasia,MOVIE,Walt Disney's timeless masterpiece is an extra...,1940,G,120,"['animation', 'family', 'music', 'fantasy']",['US'],NaN,tt0032455,7.7,94681.0,57.751,7.4
1,tm67803,Snow White and the Seven Dwarfs,MOVIE,"A beautiful girl, Snow White, takes refuge in ...",1937,G,83,"['fantasy', 'family', 'romance', 'animation', ...",['US'],NaN,tt0029583,7.6,195321.0,107.137,7.1
2,tm82546,Pinocchio,MOVIE,Lonely toymaker Geppetto has his wishes answer...,1940,G,88,"['animation', 'comedy', 'family', 'fantasy']",['US'],NaN,tt0032910,7.5,141937.0,71.160,7.1
3,tm79357,Bambi,MOVIE,Bambi's tale unfolds from season to season as ...,1942,G,70,"['animation', 'drama', 'family']",['US'],NaN,tt0034492,7.3,140406.0,68.136,7.0
4,tm62671,Treasure Island,MOVIE,Enchanted by the idea of locating treasure bur...,1950,PG,96,"['family', 'action']","['GB', 'US']",NaN,tt0043067,6.9,8229.0,10.698,6.5


In [3]:
netflix_csv_file = "./Datasets/Netflix_titles.csv"
netflix_titles_df = pd.read_csv(netflix_csv_file)
netflix_titles_df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


### Connect to local database

In [4]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'Entertainment_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

### Check for tables

In [5]:
engine.table_names()

['disney', 'netflix', 'Disney', 'Netflix']

### Use pandas to load csv converted DataFrame into database

In [6]:
disney_titles_df.to_sql(name='disney', con=engine, if_exists='append', index=False)

In [8]:
netflix_titles_df.to_sql(name='netflix', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the disney table
* NOTE: can also check using pgAdmin

In [ ]:
pd.read_sql_query('select * from disney', con=engine).head()

In [3]:
### Perform API Call
# URL for GET requests to retrieve popularity data
url = "https://api.themoviedb.org/3/movie/popular?page=1&api_key=" + tmdb_api_key
# print(url)

# Print the response object to the console
requests.get(url)

# Retreiving data and converting into JSON (Jsonify)
response = requests.get(url).json()

response

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/bQXAqRx2Fgc46uCVWgoPz5L5Dtr.jpg',
   'genre_ids': [28, 14, 878],
   'id': 436270,
   'original_language': 'en',
   'original_title': 'Black Adam',
   'overview': 'Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods—and imprisoned just as quickly—Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.',
   'popularity': 8198.814,
   'poster_path': '/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg',
   'release_date': '2022-10-19',
   'title': 'Black Adam',
   'video': False,
   'vote_average': 7.3,
   'vote_count': 2493},
  {'adult': False,
   'backdrop_path': '/7zQJYV02yehWrQN6NjKsBorqUUS.jpg',
   'genre_ids': [28, 18, 36],
   'id': 724495,
   'original_language': 'en',
   'original_title': 'The Woman King',
   'overview': 'The story of the Agojie, the all-female unit of warriors who protected the African Kingdom of Dahomey in the 1800s with skills and 

In [4]:
results_1 = response["results"]
# len(results)

tmdb_1_df = pd.DataFrame(results_1)
tmdb_1_df
    

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/bQXAqRx2Fgc46uCVWgoPz5L5Dtr.jpg,"[28, 14, 878]",436270,en,Black Adam,"Nearly 5,000 years after he was bestowed with ...",8198.814,/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg,2022-10-19,Black Adam,False,7.3,2493
1,False,/7zQJYV02yehWrQN6NjKsBorqUUS.jpg,"[28, 18, 36]",724495,en,The Woman King,"The story of the Agojie, the all-female unit o...",4662.739,/438QXt1E3WJWb3PqNniK0tAE5c1.jpg,2022-09-15,The Woman King,False,7.9,611
2,False,/au4HUSWDRadIcl9CqySlw1kJMfo.jpg,"[80, 28, 53]",829799,en,Paradise City,Renegade bounty hunter Ryan Swan must carve hi...,2645.370,/xdmmd437QdjcCls8yCQxrH5YYM4.jpg,2022-11-11,Paradise City,False,6.4,40
3,False,/sUuzl04qNIYsnwCLQpZ2RSvXA1V.jpg,"[35, 28, 53]",792775,is,Leynilögga,"When Bússi, Iceland's toughest cop, is forced ...",2308.284,/jnWyZsaCl3Ke6u6ReSmBRO8S1rX.jpg,2022-05-23,Cop Secret,False,6.3,33
4,False,/kmzppWh7ljL6K9fXW72bPN3gKwu.jpg,"[14, 28, 35, 80]",1013860,en,R.I.P.D. 2: Rise of the Damned,When Sheriff Roy Pulsipher finds himself in th...,2530.159,/g4yJTzMtOBUTAR2Qnmj8TYIcFVq.jpg,2022-11-15,R.I.P.D. 2: Rise of the Damned,False,6.7,206
5,False,/90ZZIoWQLLEXSVm0ik3eEQBinul.jpg,"[28, 27, 53]",988233,en,Hex,Following a mysterious disappearance on a jump...,2082.997,/xFJHb43ZAnnuiDztxZYsmyopweb.jpg,2022-11-01,Hex,False,5.1,13
6,False,/xDMIl84Qo5Tsu62c9DGWhmPI67A.jpg,"[28, 12, 878]",505642,en,Black Panther: Wakanda Forever,"Queen Ramonda, Shuri, M’Baku, Okoye and the Do...",1945.663,/ps2oKfhY6DL3alynlSqY97gHSsg.jpg,2022-11-09,Black Panther: Wakanda Forever,False,7.5,1205
7,False,/707thQazLJiYLBhCrZlRoV05NNL.jpg,"[28, 18, 53]",948276,fr,Balle perdue 2,"Having cleared his name, genius mechanic Lino ...",1773.923,/uAeZI1JJbLPq7Bu5dziH7emHeu7.jpg,2022-11-10,Lost Bullet 2,False,6.6,146
8,False,/jCY35GkjwWUmoPO9EV1lWL6kuyj.jpg,"[28, 12, 53]",855440,es,Polar,"MG, a policewoman who has been expelled from t...",1731.526,/efuKHH9LqBZB67AS87kprLgaYO8.jpg,2022-10-26,Polar,False,7.5,2
9,False,/vmDa8HijINCAFYKqsMz0YM3sVyE.jpg,"[80, 28, 53]",747803,en,One Way,On the run with a bag full of cash after a rob...,1708.892,/uQCxOziq79P3wDsRwQhhkhQyDsJ.jpg,2022-09-02,One Way,False,6.4,21


In [5]:
tmdb_1_df = tmdb_1_df.set_index('id')
tmdb_1_df

,adult,backdrop_path,genre_ids,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
id,,,,,,,,,,,,,
436270,False,/bQXAqRx2Fgc46uCVWgoPz5L5Dtr.jpg,"[28, 14, 878]",en,Black Adam,"Nearly 5,000 years after he was bestowed with ...",8198.814,/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg,2022-10-19,Black Adam,False,7.3,2493
724495,False,/7zQJYV02yehWrQN6NjKsBorqUUS.jpg,"[28, 18, 36]",en,The Woman King,"The story of the Agojie, the all-female unit o...",4662.739,/438QXt1E3WJWb3PqNniK0tAE5c1.jpg,2022-09-15,The Woman King,False,7.9,611
829799,False,/au4HUSWDRadIcl9CqySlw1kJMfo.jpg,"[80, 28, 53]",en,Paradise City,Renegade bounty hunter Ryan Swan must carve hi...,2645.370,/xdmmd437QdjcCls8yCQxrH5YYM4.jpg,2022-11-11,Paradise City,False,6.4,40
792775,False,/sUuzl04qNIYsnwCLQpZ2RSvXA1V.jpg,"[35, 28, 53]",is,Leynilögga,"When Bússi, Iceland's toughest cop, is forced ...",2308.284,/jnWyZsaCl3Ke6u6ReSmBRO8S1rX.jpg,2022-05-23,Cop Secret,False,6.3,33
1013860,False,/kmzppWh7ljL6K9fXW72bPN3gKwu.jpg,"[14, 28, 35, 80]",en,R.I.P.D. 2: Rise of the Damned,When Sheriff Roy Pulsipher finds himself in th...,2530.159,/g4yJTzMtOBUTAR2Qnmj8TYIcFVq.jpg,2022-11-15,R.I.P.D. 2: Rise of the Damned,False,6.7,206
988233,False,/90ZZIoWQLLEXSVm0ik3eEQBinul.jpg,"[28, 27, 53]",en,Hex,Following a mysterious disappearance on a jump...,2082.997,/xFJHb43ZAnnuiDztxZYsmyopweb.jpg,2022-11-01,Hex,False,5.1,13
505642,False,/xDMIl84Qo5Tsu62c9DGWhmPI67A.jpg,"[28, 12, 878]",en,Black Panther: Wakanda Forever,"Queen Ramonda, Shuri, M’Baku, Okoye and the Do...",1945.663,/ps2oKfhY6DL3alynlSqY97gHSsg.jpg,2022-11-09,Black Panther: Wakanda Forever,False,7.5,1205
948276,False,/707thQazLJiYLBhCrZlRoV05NNL.jpg,"[28, 18, 53]",fr,Balle perdue 2,"Having cleared his name, genius mechanic Lino ...",1773.923,/uAeZI1JJbLPq7Bu5dziH7emHeu7.jpg,2022-11-10,Lost Bullet 2,False,6.6,146
855440,False,/jCY35GkjwWUmoPO9EV1lWL6kuyj.jpg,"[28, 12, 53]",es,Polar,"MG, a policewoman who has been expelled from t...",1731.526,/efuKHH9LqBZB67AS87kprLgaYO8.jpg,2022-10-26,Polar,False,7.5,2


In [7]:
tmdb_1_df.rename(columns={'popularity':'tmdb_popularity', 'overview':'description', 'vote_average':'tmdb_score' }, inplace = True)
tmdb_1_df

,adult,backdrop_path,genre_ids,original_language,original_title,description,tmdb_popularity,poster_path,release_date,title,video,tmdb_score,vote_count
id,,,,,,,,,,,,,
436270,False,/bQXAqRx2Fgc46uCVWgoPz5L5Dtr.jpg,"[28, 14, 878]",en,Black Adam,"Nearly 5,000 years after he was bestowed with ...",8198.814,/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg,2022-10-19,Black Adam,False,7.3,2493
724495,False,/7zQJYV02yehWrQN6NjKsBorqUUS.jpg,"[28, 18, 36]",en,The Woman King,"The story of the Agojie, the all-female unit o...",4662.739,/438QXt1E3WJWb3PqNniK0tAE5c1.jpg,2022-09-15,The Woman King,False,7.9,611
829799,False,/au4HUSWDRadIcl9CqySlw1kJMfo.jpg,"[80, 28, 53]",en,Paradise City,Renegade bounty hunter Ryan Swan must carve hi...,2645.370,/xdmmd437QdjcCls8yCQxrH5YYM4.jpg,2022-11-11,Paradise City,False,6.4,40
792775,False,/sUuzl04qNIYsnwCLQpZ2RSvXA1V.jpg,"[35, 28, 53]",is,Leynilögga,"When Bússi, Iceland's toughest cop, is forced ...",2308.284,/jnWyZsaCl3Ke6u6ReSmBRO8S1rX.jpg,2022-05-23,Cop Secret,False,6.3,33
1013860,False,/kmzppWh7ljL6K9fXW72bPN3gKwu.jpg,"[14, 28, 35, 80]",en,R.I.P.D. 2: Rise of the Damned,When Sheriff Roy Pulsipher finds himself in th...,2530.159,/g4yJTzMtOBUTAR2Qnmj8TYIcFVq.jpg,2022-11-15,R.I.P.D. 2: Rise of the Damned,False,6.7,206
988233,False,/90ZZIoWQLLEXSVm0ik3eEQBinul.jpg,"[28, 27, 53]",en,Hex,Following a mysterious disappearance on a jump...,2082.997,/xFJHb43ZAnnuiDztxZYsmyopweb.jpg,2022-11-01,Hex,False,5.1,13
505642,False,/xDMIl84Qo5Tsu62c9DGWhmPI67A.jpg,"[28, 12, 878]",en,Black Panther: Wakanda Forever,"Queen Ramonda, Shuri, M’Baku, Okoye and the Do...",1945.663,/ps2oKfhY6DL3alynlSqY97gHSsg.jpg,2022-11-09,Black Panther: Wakanda Forever,False,7.5,1205
948276,False,/707thQazLJiYLBhCrZlRoV05NNL.jpg,"[28, 18, 53]",fr,Balle perdue 2,"Having cleared his name, genius mechanic Lino ...",1773.923,/uAeZI1JJbLPq7Bu5dziH7emHeu7.jpg,2022-11-10,Lost Bullet 2,False,6.6,146
855440,False,/jCY35GkjwWUmoPO9EV1lWL6kuyj.jpg,"[28, 12, 53]",es,Polar,"MG, a policewoman who has been expelled from t...",1731.526,/efuKHH9LqBZB67AS87kprLgaYO8.jpg,2022-10-26,Polar,False,7.5,2


In [ ]:
# Rename the columns to match the disney and netflix column names
tmdb_1_df.to_sql(name='tmdb', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from tmdb', con=engine).head()